In [2]:
import pandas as pd
import numpy as np

In [ ]:
filename = 'ANONYMOUS_DATA_202210_60002'
df = pd.read_csv(f'../data/raw_data_unzipped/{filename}.csv',)

In [3]:
# remove rows with missing values
hourly_cols = df.iloc[:, 7:55]
df = df[~hourly_cols.isna().any(axis=1)]

In [4]:
df['INTERVAL_READING_DATE'] = pd.to_datetime(df['INTERVAL_READING_DATE'])
df['DAY'] = df['INTERVAL_READING_DATE'].dt.day

In [5]:
# keep only full months users
user_date_counts = (
    df.groupby('ACCOUNT_IDENTIFIER')['INTERVAL_READING_DATE']
    .count()
    .reset_index(name='DATE_COUNT')
)

full_month_users = (
    user_date_counts
    .query('DATE_COUNT == 31')
    ['ACCOUNT_IDENTIFIER']
    .tolist()

)

df = (
    df[df.ACCOUNT_IDENTIFIER.isin(full_month_users)]
)


In [6]:
reshaped_user_data = []
time_interval_columns = [
    f"INTERVAL_HR{(30 * i) // 60:02d}{(30 * i) % 60:02d}_ENERGY_QTY"
    for i in range(1, 49)
]

for user in full_month_users:
    user_df = (
        df.query('ACCOUNT_IDENTIFIER == @user')
        .sort_values('DAY')
    )
    reshaped_user_data.append(
        user_df[['DAY']+time_interval_columns].values.reshape(-1, 1)
    )
reshaped_user_data = np.hstack(reshaped_user_data)

np.save(f'{filename}.npy', reshaped_user_data)


In [7]:
reshaped_user_data

array([[1.    , 1.    , 1.    , ..., 1.    , 1.    , 1.    ],
       [0.1575, 0.4025, 0.145 , ..., 0.0825, 1.0707, 0.3275],
       [0.1175, 0.45  , 0.135 , ..., 0.075 , 1.1607, 0.315 ],
       ...,
       [0.1225, 0.5225, 0.235 , ..., 0.2325, 1.0857, 0.1575],
       [0.13  , 0.6   , 0.23  , ..., 0.2625, 0.8735, 0.1   ],
       [0.1575, 0.4075, 0.0775, ..., 0.295 , 0.8782, 0.1675]],
      shape=(1519, 11519))

In [8]:
time_interval_columns = [
    f"INTERVAL_HR{(30 * i) // 60:02d}{(30 * i) % 60:02d}_ENERGY_QTY"
    for i in range(1, 49)
]

In [9]:
interval_columns

NameError: name 'interval_columns' is not defined

In [22]:
A = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 22, 23, 24],
    [25, 26, 27, 28],
    [29, 30, 31, 32]
])

In [12]:
A[::2, :]

array([[ 1,  2,  3,  4],
       [ 9, 10, 11, 12],
       [17, 18, 19, 20],
       [25, 26, 27, 28]])

In [ ]:

hour=2
step = int(2*hour)
row_size, col_size = int(A.shape[0] // step), int(A.shape[1])
A_hour=np.zeros((row_size, col_size))

for i in range(step):
    A_hour += A[i::step, :]


array([[ 28.,  32.,  36.,  40.],
       [ 92.,  96., 100., 104.]])

In [21]:
A.shape[1] // hour

8.0

In [30]:
int(A.shape[0] // hour)

16

In [24]:
A=A[:,0].reshape(-1, 1)


In [25]:
r=3
data_with_previous_r_intervals = []
for i in range(A.shape[1]):
    col_with_r_intervals = []
    for j in range(A.shape[0]-r):
        col_with_r_intervals.append(A[j:j+r, i].reshape(1, -1))
    col_with_r_intervals = np.vstack(col_with_r_intervals)
    data_with_previous_r_intervals.append(col_with_r_intervals)
data_with_previous_r_intervals = np.hstack(data_with_previous_r_intervals)
data_with_previous_r_intervals

array([[ 1,  5,  9],
       [ 5,  9, 13],
       [ 9, 13, 17],
       [13, 17, 21],
       [17, 21, 25]])

In [29]:
np.concat([data_with_previous_r_intervals, A[r:,:]], axis=1)

array([[ 1,  5,  9, 13],
       [ 5,  9, 13, 17],
       [ 9, 13, 17, 21],
       [13, 17, 21, 25],
       [17, 21, 25, 29]])

In [28]:
A[r,:]

array([13])

In [30]:
np.arange(20)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from src.exception import CustomException
from src.logger import logging
from dataclasses import dataclass
from catboost import CatBoostRegressor
from sklearn.ensemble import (
    AdaBoostRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from src.utils import save_object, evaluate_models


models = {
    "Random Forest": RandomForestRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "Linear Regression": LinearRegression(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}

In [20]:
test = ["best"]
test["best"]

TypeError: list indices must be integers or slices, not str

In [11]:
models["Random Forest"]

RandomForestRegressor()

In [1]:
strategy_name={"rma_pca", "rma_rbd"}

for strategy in strategy_name:
    print(strategy)

rma_rbd
rma_pca
